In [ ]:
%matplotlib widget

import polars as pl
import numpy as np

import k3d
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

from pyergonomics import ProjectSettings, add_pose_assessment_columns
from pose_skeletons import get_skeleton_def # TODO move to project settings

In [ ]:
#
# 1. load data
#
# this file is capturing me using the optitrack of humlab at Umea.
# one person, clean and nice data
#

settings = ProjectSettings("../data/optitrack-z")
skeleton = get_skeleton_def("optitrack") # TODO put in project settings

print(settings)

In [ ]:
#
# visualize skeleton
#

def set_frame(n):
    joints = settings.tracker.get_keypoints_for_person(person_id, frame = n)[0]
    joints_plot.positions = joints
    lines_plot.vertices = joints

person_id = settings.tracker.get_person_ids()[0]

plot = k3d.plot(grid=(-2, -2, 0, 2, 2, 2), camera_mode='orbit')

joints_plot = k3d.points([], point_size=0.03, shader='flat', color=0xff0000) 
lines_plot = k3d.lines([], list(skeleton.bones), width=0.005, color=0xff0000, indices_type="segment")

plot += joints_plot
plot += lines_plot
plot.display()

interact(set_frame, n=widgets.IntSlider(min=0, max=settings.number_of_frames-1, step=1, value=10, layout=widgets.Layout(width='100%')))

In [ ]:
#
# make assessment
#

add_pose_assessment_columns(settings.tracker, skeleton)

In [ ]:
#
# plot pose assesment
#

person_id = settings.tracker.get_person_ids()[0]
metrics = settings.tracker.get_pose_metrics_for_person(person_id)
time_axis = np.arange(settings.number_of_frames) / settings.frames_per_second

plt.figure(figsize=(10, 5))

plt.plot(time_axis, metrics["trunk_bending"], label="Bending")
plt.plot(time_axis, metrics["trunk_side_bending"], label="Side Bending")
plt.plot(time_axis, metrics["trunk_twist"], label="Twist")

plt.axhline(y=60, color='#ff9999', linestyle='--', alpha=0.5, label="Threshold (60°)")
plt.axhline(y=20, color='#ffcc99', linestyle='--', alpha=0.5, label="Threshold (20°)")

plt.xlabel("Time (seconds)")  # Update the label
plt.ylabel("Angle (degrees)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
#
# plot aggregated data
#

bending_data = metrics["trunk_bending"]

count_low = np.sum(bending_data <= 20)    
count_medium = np.sum((bending_data > 20) & (bending_data < 60))
count_high = np.sum(bending_data >= 60)

# 3. Plot Pie Chart
sizes = [count_low, count_medium, count_high]
labels = ['Up to 20°', '20° - 60°', '60° and above']
colors = ['#99ff99', '#ffcc99', '#ff9999']  # Green, Orange, Red

plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title(f"Trunk Bending Distribution\n(Total Seconds: {len(bending_data)/settings.frames_per_second:.2f})")
plt.axis('equal')
plt.show()

In [ ]:
#
# combine 3D with 3D
#


